In [1]:
# %run "Scripts/dataprep.py"
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Codigos\\Análise de BCI\\Scripts')

In [2]:
from classes import *

In [12]:
firstChannels = 5
fixedSizeOutChannels = 32
layerSize = 5

inSizes = [firstChannels, fixedSizeOutChannels]
        
for i in range(layerSize - 2):
    inSizes.append(inSizes[-1] + fixedSizeOutChannels)

In [41]:
def ConvBlocks(inSizes, outSizes, middleActivationFunc = 'relu', *args, **kwargs):
    return nn.Sequential(*[ConvBlock(inputSize, outputSize, middleActivationFunc, *args, **kwargs) for
                                     inputSize, outputSize in zip(inSizes, outSizes)])

class DenseBlock(nn.Module):
    def __init__(self, firstChannels, fixedSizeOutChannels, layerSize, kernel_size):
        super(DenseBlock, self).__init__()
        
        inSizes = [firstChannels, fixedSizeOutChannels]
        outSizes = [fixedSizeOutChannels]*layerSize
        
        for i in range(layerSize - 2):
            inSizes.append(inSizes[-1] + fixedSizeOutChannels)
            
        self.BN = nn.BatchNorm2d(num_features = firstChannels)
        self.ConvLayers = ConvBlocks(inSizes, outSizes, 
                                     kernel_size = kernel_size, stride = 1, padding = kernel_size//2)
    
    def forward(self, X):
        X = self.BN(X)
        output = [self.ConvLayers[0](X)]
        dense  = nn.ReLU(torch.cat(output, 1))
        for conv in self.ConvLayers[1:]:
            Y = conv(dense)
            output.append(Y)
            dense = nn.ReLU(torch.cat(output, 1))
            
        return dense    
    
class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        
        self.relu = nn.ReLU(inplace = True)
        self.bn   = nn.BatchNorm2d(num_features = out_channels)
        self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                              kernel_size = 1, bias = False)
        self.avg_pool = nn.AvgPool2d(kernel_size = 2, stride = 2, padding = 0)
        
    def forward(self, X):
        bn = self.bn(self.relu(self.conv(X)))
        out = self.avg_pool(bn)
        
        return out
    
    
class DenseNet(nn.Module):
    def __init__(self, nr_classes):
        super(DenseNet, self).__init__()
        
        self.FirstConv = nn.Sequential(
                nn.ReLU(),
                nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, bias = False)
        )
        
        self.DenseLayer1 = nn.Sequential( # 160 = 32 * 5
                DenseBlock(64, 32, 5, 3),
                TransitionLayer(160, 128) 
        )
        self.DenseLayer2 = nn.Sequential(
                DenseBlock(128, 32, 5, 3),
                TransitionLayer(160, 128)
        )
        self.DenseLayer3 = nn.Sequential(
                DenseBlock(128, 32, 5, 3),
                TransitionLayer(160, 64)
        )
        self.BN = nn.BatchNorm2d(num_features = 64)
        self.Classifier = nn.Sequential(
                nn.Linear(64*4*4, 512),
                nn.Linear(512, nr_classes)
        )
        
    def forward(self, X):
        X = self.FirstConv(X)
        X = self.DenseLayer1(X)
        X = self.DenseLayer2(X)
        X = self.DenseLayer3(X)
        X = self.BN(X)
        x.view(-1, 64*4*4)
        X = self.Classifier(X)
        return X

In [43]:
DenseNet(5)

DenseNet(
  (FirstConv): Sequential(
    (0): ReLU()
    (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  )
  (DenseLayer1): Sequential(
    (0): DenseBlock(
      (BN): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ConvLayers): Sequential(
        (0): Sequential(
          (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (2): Sequential(
          (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

In [20]:
class Dense_Block(nn.Module):
    # Faça uma classe geral disso
    def __init__(self, in_channels):
        super(Dense_Block, self).__init__()
        
        self.relu = nn.ReLU(inplace = True)
        self.bn   = nn.BatchNorm2d(num_features = in_channels)
        
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 32,
                               kernel_size = 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32,
                               kernel_size = 3, stride = 1, padding = 1)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 32, 
                               kernel_size = 3, stride = 1, padding = 1)
        self.conv4 = nn.Conv2d(in_channels = 96, out_channels = 32,
                               kernel_size = 3, stride = 1, padding = 1)
        self.conv5 = nn.Conv2d(in_channels = 128, out_channels = 32,
                               kernel_size = 3, stride = 1, padding = 1)
        
    def forward(self, X):
        bn = self.bn(X)
        conv1 = self.relu(self.conv1(bn))

        conv2 = self.relu(self.conv2(conv1))
        # Concatenate in channel dimension
        c2_dense = self.relu(torch.cat([conv1, conv2], 1))

        conv3 = self.relu(self.conv3(c2_dense))
        c3_dense = self.relu(torch.cat([conv1, conv2, conv3], 1))

        conv4 = self.relu(self.conv4(c3_dense))
        c4_dense = self.relu(torch.cat([conv1, conv2, conv3, conv4], 1))

        conv5 = self.relu(self.conv5(c4_dense))
        c5_dense = self.relu(torch.cat([conv1, conv2, conv3, conv4, conv5], 1))

        return c5_dense
    
class Transition_Layer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Transition_Layer, self).__init__()
        
        self.relu = nn.ReLU(inplace = True)
        self.bn   = nn.BatchNorm2d(num_features = out_channels)
        self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                              kernel_size = 1, bias = False)
        self.avg_pool = nn.AvgPool2d(kernel_size = 2, stride = 2, padding = 0)
        
    def forward(self, X):
        bn = self.bn(self.relu(self.conv(X)))
        out = self.avg_pool(bn)
        
        return out
    
class DenseNet(nn.Module):
    def __init__(self, nr_classes):
        super(DenseNet, self).__init__()
        
        self.lowconv = nn.Conv2d(in_channels = 3, out_channels = 64, 
                                 kernel_size = 3, bias = False)
        self.relu = nn.ReLU()
        
        # Dense Blocks
        self.denseblock1 = self._make_dense_block(Dense_Block, 64)
        self.denseblock2 = self._make_dense_block(Dense_Block, 128)
        self.denseblock3 = self._make_dense_block(Dense_Block, 128)
        
        # Make transition Layers
        self.transitionLayer1 = self._make_transition_layer(Transition_Layer,
                                                            in_channels = 160, out_channels = 128)
        self.transitionLayer2 = self._make_transition_layer(Transition_Layer,
                                                            in_channels = 160, out_channels = 128)
        self.transitionLayer3 = self._make_transition_layer(Transition_Layer,
                                                            in_channels = 160, out_channels = 64)
        
        # Classifier
        self.bn = nn.BatchNorm2d(num_features = 64)
        self.pre_classifier = nn.Linear(64*4*4, 512)
        self.classifier = nn.Linear(512, nr_classes)
        
    def _make_dense_block(self, block, in_channels):
        layers = []
        layers.append(block(in_channels))
        return nn.Sequential(*layers)
    
    def _make_transition_layer(self, layer, in_channels, out_channels):
        modules = []
        modules.append(layer(in_channels, out_channels))
        return nn.Sequential(*modules)
    
    def forward(self, X):
        out = self.relu(self.lowconv(X))
        
        out = self.denseblock1(out)
        out = self.transitionLayer1(out)
        
        out = self.denseblock2(out)
        out = self.transitionLayer2(out)
        
        out = self.denseblock3(out)
        out = self.transitionLayer3(out)
        
        out = self.bn(out)
        out = out.view(-1, 64*4*4)
        
        out = self.pre_classifier(out)
        out = self.classifier(out)
        
        return out